In [1]:
from pandas import read_csv
df = read_csv("adult.data.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Допустим стоит задача сделать классификатор, который будет определять какая зп у человека с заданными параметрами: образования, пол, возраст и расса

In [2]:
#подготовим данные

df = df[['salary', 'education', 'sex', 'age', 'race']]

from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame

def encodeSeries(df, label):
    label_encoder = LabelEncoder()
    encoded = label_encoder.fit_transform(df[label].values)
    return encoded
    
#закодируем строковые данные
newData = {'salary': encodeSeries(df, 'salary'), 'age': df['age'].values, 'education': encodeSeries(df, 'education'), 
           'sex': encodeSeries(df, 'sex'), 'race': encodeSeries(df, 'race')}
df = DataFrame(newData)
df = df.reindex(columns=['salary', 'education', 'age', 'sex', 'race'])
df.head()

,salary,education,age,sex,race
0,0,9,39,1,4
1,0,9,50,1,4
2,0,11,38,1,4
3,0,1,53,1,2
4,0,9,28,0,2


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

y = df['salary']
X_train, X_holdout, y_train, y_holdout = train_test_split(df.values, y, test_size=0.3,
                                                          random_state=17)
tree = DecisionTreeClassifier(max_depth=5, random_state=17)
knn = KNeighborsClassifier(n_neighbors=10)

In [4]:
%%time
tree.fit(X_train, y_train)

CPU times: user 6.13 ms, sys: 2.17 ms, total: 8.3 ms
Wall time: 7.17 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [5]:
%%time
knn.fit(X_train, y_train)

CPU times: user 26.7 ms, sys: 1.61 ms, total: 28.3 ms
Wall time: 28.8 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

Качество прогнозов будем проверять с помощью простой метрики – доли правильных ответов

Сделаем прогнозы для отложенной выборки. Видим, что метод ближайших соседей справился намного лучше. Но это мы пока выбирали параметры наугад.

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
tree_pred = tree.predict(X_holdout)
accuracy_score(y_holdout, tree_pred)

1.0

In [8]:
knn_pred = knn.predict(X_holdout)
accuracy_score(y_holdout, knn_pred)

0.9771726891186406

Теперь настроим параметры дерева на кросс-валидации. Настраивать будем максимальную глубину и максимальное используемое на каждом разбиении число признаков. Суть того, как работает GridSearchCV: для каждой уникальной пары значений параметров max_depth и max_features будет проведена 5-кратная кросс-валидация и выберется лучшее сочетание параметров.

In [9]:
#проблемы с кодировкой
import sys
reload(sys)
sys.setdefaultencoding('utf8')

from sklearn.model_selection import GridSearchCV, cross_val_score
tree_params = {'max_depth': range(1,11),
               'max_features': range(1,5)}
tree_grid = GridSearchCV(tree, tree_params,
                         cv=5, n_jobs=-1,
                        verbose=True)
tree_grid.fit(X_train, y_train)

In [10]:
GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(1, 11), 'max_features': range(1, 5)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': [1, 2, 3, 4], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [11]:
tree_grid.best_params_

{'max_depth': 1, 'max_features': 3}

In [12]:
tree_grid.best_score_

1.0

In [13]:
accuracy_score(y_holdout, tree_grid.predict(X_holdout))

1.0

Теперь попробуем настроить число соседей в алгоритме kNN.

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])
knn_params = {'knn__n_neighbors': range(1, 10)}
knn_grid = GridSearchCV(knn_pipe, knn_params,
                         cv=5, n_jobs=-1,
                        verbose=True)
knn_grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [15]:
knn_grid.best_params_, knn_grid.best_score_

({'knn__n_neighbors': 1}, 1.0)

In [16]:
accuracy_score(y_holdout, knn_grid.predict(X_holdout))

1.0